# Tools for Analyzing Time Series of Satellite Imagery (TATSSI)

## Download data from the GEE
TATSSI allows to dowload data products from Google Earth Engine [GEE](https://developers.google.com/earth-engine/datasets/) and create and process time series from different sensors such as:

* Moderate Resolution Imaging Spectroradiometer [MODIS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#MODIS_anchor)
* Visible Infrared Imaging Radiometer Suite [VIIRS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#S-NPPVIIRS_anchor)

Downloading products from the GEE requires a Google Account, please, first regsiter as a user to get the login credentials and follow the instructions to authenticate to the Earth Engine using the [Python API](https://developers.google.com/earth-engine/python_install).

1. Select your are of study, you can save it as a GeoJSON file, e.g. from the [geojson.io](http://geojson.io/) site.
2. For the time being you need to match the official MODIS product bands and the GEE bands since the band names are not consistent.
3. Generate the time series, you can then use the rest of the (awesome) TATSSI tools!

In [1]:
# Import libraries
import os
import json

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from helpers import utils
from TATSSI.download.gee_downloader import export_collection
from TATSSI.time_series.generator import Generator

In [2]:
# Area of study
with open('bolivia.geojson') as json_file:
    region = json.load(json_file)
    
region = region['features'][0]['geometry']['coordinates']
geometry = ee.Geometry.Polygon(region)
print(geometry)

ee.Geometry({
  "type": "Polygon",
  "coordinates": [
    [
      [
        -64.1107177734375,
        -20.535077326962796
      ],
      [
        -63.02856445312499,
        -20.535077326962796
      ],
      [
        -63.02856445312499,
        -19.580493479202513
      ],
      [
        -64.1107177734375,
        -19.580493479202513
      ],
      [
        -64.1107177734375,
        -20.535077326962796
      ]
    ]
  ],
  "evenOdd": true
})


For this example we will use the MODIS MOD13A2 product. Match the band names from the [LPDAAC](https://lpdaac.usgs.gov/products/mod13a2v006/) and the [GEE](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2).

In [3]:
# Dictionary with "GEE" : "LPDAAC"
bands = {"DayOfYear" : "1_km_16_days_composite_day_of_the_year",
         "EVI" : "1_km_16_days_EVI",
         "RelativeAzimuth" : "1_km_16_days_relative_azimuth_angle",
         "SummaryQA" : "1_km_16_days_pixel_reliability",
         "sur_refl_b02" : "1_km_16_days_NIR_reflectance",
         "sur_refl_b07" : "1_km_16_days_MIR_reflectance",
         "DetailedQA" : "1_km_16_days_VI_Quality",
         "NDVI" : "1_km_16_days_NDVI",
         "SolarZenith" : "1_km_16_days_sun_zenith_angle",
         "sur_refl_b01" : "1_km_16_days_red_reflectance",
         "sur_refl_b03" : "1_km_16_days_blue_reflectance",
         "ViewZenith" : "1_km_16_days_view_zenith_angle"}

Set the download parameters

In [4]:
# Dates as YYYY-MM-DD
start_date, end_date = '2017-06-01', '2020-06-30'

# Get MODIS data
product_name = 'MOD13A2'
version = '006'

# Product using GEE nomenclature
product = f'MODIS/{version}/{product_name}'

# Get GEE collection
ee_collection = (ee.ImageCollection(product).
                 filterBounds(geometry).
                 filterDate(start_date, end_date))

# Output directory
output_dir = '/tmp/bolivia'

Get the data!

In [5]:
datasets = export_collection(ee_collection, output_dir, bands=bands,
                             product=product, version=version,
                             scale=None, crs=None, region=region,
                             file_per_band=True)

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Total number of images: 70

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 1/70: 2017_06_10.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f1fdb06f9810f33edc68b97e9968fdab-54c6574dc620e82987abba8bcf924e5a:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /tmp/bolivia
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 2/70: 2017_06_26.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/82e60365a7b464651aeacde200175d88-b010acd339f3ed1e4e8309fadf9bb0f4:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /tmp/bolivia
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 3/70: 2017_07_12.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


...

INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9309bf95a828cc76345497d16f4ab7ee-3ec95ddcd84127d2671c691bc7a08b60:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /tmp/bolivia
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 69/70: 2020_05_24.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/86e23e14bd59344cb10b85ec9fd6ac0d-f84e897fd34e5520140df6535c93bddf:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /tmp/bolivia
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 70/70: 2020_06_09.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/289e90b505fe0b64ba6fd51ca6c50481-f38e43e0155849eb94c562f252894623:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /tmp/bolivia


Create TATSSI time series.

In [6]:
# TATSSI Time Series Generator object
tsg = Generator(source_dir=output_dir, product=product_name,
                version=version, data_format='tif',
                progressBar=None, preprocessed=True)

# Create layerstacks
tsg._Generator__datasets = datasets
for dataset in tsg._Generator__datasets:
    tsg._Generator__generate_layerstack(dataset, 'tif')

# For the associated product layers, decode the corresponding QA bands
tsg._Generator__decode_qa('tif')

INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_NDVI created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_EVI created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_VI_Quality created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_red_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_NIR_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_blue_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_MIR_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_view_zenith_angle created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_sun_zenith_angle created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_relative_azimuth_angle created successfully.
INFO:TA

INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2017_11_01.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.qua

INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2018_04_07.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2018_04_23.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:

INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2018_09_30.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...

INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2019_03_06.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask..

INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2019_08_13.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.qua

INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2020_01_17.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_VI_Quality/MOD13A2.2020_02_02.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:

INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2017_06_26.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2017_07_12.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2017_07_28.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding M

INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2018_09_30.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2018_10_16.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2018_11_01.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding M

INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2020_01_01.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2020_01_17.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /tmp/bolivia/1_km_16_days_pixel_reliability/MOD13A2.2020_02_02.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding M